In [1]:
import numpy as np
import dicom
import sys
sys.path.append('..')
import os
from AlgoEngine.utils import getContours, getImageBlock, convertROIToCTSpace
from AlgoEngine.ovh import getOVHDistances
from General.testing_utils import getContourInputs
from math import sqrt

## Inputs to Function

We compute OVH distances for PTV and bladder in this test

In [2]:
BASE_DIR = '/home/radiation/RadiationTherapyDecisionSupport/data/'
StudyID = 'UCLA_PR_6'
_, sop_ids = getImageBlock(StudyID, BASE_DIR)

ctFilenames = [fl for fl in os.listdir(BASE_DIR + StudyID) if 'CT.' in fl]
numImages = len(ctFilenames)

sampleCTImage = dicom.read_file(BASE_DIR + StudyID + '/' + ctFilenames[0])
width = sampleCTImage.Columns
height = sampleCTImage.Rows
row_spacing = float(sampleCTImage.PixelSpacing[0])
column_spacing = float(sampleCTImage.PixelSpacing[1])
slice_spacing = float(sampleCTImage.SliceThickness)

In [3]:
ROI_NAME = 'PTV'
block_shape, contour_data, image_orientation, image_position, pixel_spacing = getContourInputs(BASE_DIR, StudyID, ROI_NAME, excluding=[])
ptv_contour_block, ptv_roi_block = getContours(block_shape, contour_data, image_orientation, image_position, pixel_spacing)
ptv_contour_block = convertROIToCTSpace(ptv_contour_block, image_position, sop_ids)
ptv_roi_block = convertROIToCTSpace(ptv_roi_block, image_position, sop_ids)

In [4]:
ROI_NAME = 'Bladder'
block_shape, contour_data, image_orientation, image_position, pixel_spacing = getContourInputs(BASE_DIR, StudyID, ROI_NAME, excluding=[])
_, oar_roi_block = getContours(block_shape, contour_data, image_orientation, image_position, pixel_spacing)
oar_roi_block = convertROIToCTSpace(oar_roi_block, image_position, sop_ids)

## Function Starts Here

In [5]:
oar_dists = getOVHDistances(oar_roi_block, ptv_contour_block, ptv_roi_block, row_spacing, column_spacing, slice_spacing)

## Testing Output

Note that because we lack access to an actual validation example, we test using the assertion that for 5 random places in the OAR, there is not a place in the PTV that is closer than the marked distance. 

In [6]:
oar_row, oar_col, oar_slice = np.nonzero(oar_dists)
ptv_row, ptv_col, ptv_slice = np.nonzero(ptv_contour_block)

alpha = column_spacing / row_spacing
beta =  slice_spacing / row_spacing

test_passed = True

indices = np.arange(oar_row.shape[0])
np.random.shuffle(indices)

for n in range(0, 30):
    oar_point = (oar_row[indices[n]], oar_col[indices[n]], oar_slice[indices[n]])
    for i in range(0, ptv_row.shape[0]):
        dist = sqrt( (oar_point[0] - ptv_row[i])**2  + 
                      alpha*(oar_point[1] - ptv_col[i])**2 + 
                      beta*(oar_point[2] - ptv_slice[i])**2)
        
        # allclose handles errors from comparing single precision to double precision number
        if dist < oar_dists[oar_point] and not np.allclose(np.array(dist), np.array(oar_dists[oar_point])):
            test_passed = False

print(test_passed)

True
